In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Import necessary data sets
circuits = pd.read_csv("/Users/lydiaking/Documents/GitHub/qtm151spring2025/final_project/data_raw/circuits.csv")
races = pd.read_csv("/Users/lydiaking/Documents/GitHub/qtm151spring2025/final_project/data_raw/races.csv")
drivers = pd.read_csv("/Users/lydiaking/Documents/GitHub/qtm151spring2025/final_project/data_raw/drivers.csv")
results = pd.read_csv("/Users/lydiaking/Documents/GitHub/qtm151spring2025/final_project/data_raw/results.csv")



#Figuring out the years for each dataset
# Merge results with races to get the year for results
results_with_year = pd.merge(results, races[['raceId', 'year']], on='raceId')

# Merge drivers with results_with_year to get the year for drivers
drivers_with_year = pd.merge(drivers, results_with_year[['driverId', 'year']].drop_duplicates(), on='driverId')

# Merge circuits with races to get the year for circuits
circuits_with_year = pd.merge(circuits, races[['circuitId', 'year']].drop_duplicates(), on='circuitId')

# Display the years for each dataset
print("Years for drivers dataset:", drivers_with_year['year'].unique())
print("Years for results dataset:", results_with_year['year'].unique())
print("Years for races dataset:", races['year'].unique())
print("Years for circuits dataset:", circuits_with_year['year'].unique())

Description of the datasets: 


The drivers dataset contains relevant information on each of the F1 drivers. Each row represents an individual driver. There are 857 observations. It spans from 1950 to 2008. The 'driverId; is each driver's unique numeric identifier, while the 'driverRef' is a unique categorical identifier (driver last name with no caps). The 'number' is their jersey number, and the 'code' is a shortened version of their last name. The 'forename' is their first name, and the 'surname' is their last name. The 'dob' variable is their date of birth, and their 'nationality' indicates the country where they are from. 

The results dataset has all the metrics for each driver's race performance. Each row represents an individual driver's specific results for a certain race. There are 25, 840 observations. It spans from 1950 to 2022. It has the 'resultId', which is the unique numeric identifier for a specific result, as well as 'raceId', which is the unique numeric identifier for a specific race. It also includes 'driverId' (explained above). It has 'constructorId', which is the unique numeric identifier for each company that makes F1 cars. 

The races dataset has relevant information on specific races that have occurred. Each row represents a specific race that occurred at a specific time and place. There are 1,102 observations. It has the 'raceId', which is the unique number assigned to each race. It also has the 'year' variable, which is the year in which the race took place. The 'round' variable refers to which race out of the total races in the season calendar it is. The 'circuitId' was explained above. The 'name' variable is the full name of the circuit. The 'date' is the date in YYYY-MM-DD format, in which the race occurred. The 'time' variable is when the race started, the fp variables refer to the date and time when practice runs occurred; the quali variables refer to the date and time when qualifiers occurred; the sprint variables refer to the date and time when sprints occurred. 

The circuits variable has relevant information on the circuits (the race tracks). Each row represents a specific circuit. There are 77 observations. The circuits variable has the 'circuitId', which is each circuit's unique numeric identifier, and the 'circuitRef', which is each circuit's categorical identifier. It has the 'name', which is the full name of the circuit, as well as 'location', which is the city where the circuit is. It also has the 'country' variable, which is the country where the circuit is located, as well as 'lat', 'lng', and 'alt', which are the latitude, longitude, and altitude (meters) of the circuit, respectively. It also has 'number' (explained above). 'Grid' refers to each driver's starting grid position, and 'position' refers to the official classification of the driver's end position (can include null, if they haven't finished). 'positionText' can include information for if a driver retired or is disqualified. 'positionOrder' is a pure number that sorts driver in order of finish, even if they never finished the course. 'points' refers to the points that the driver gets at the end of race (certain points are assigned certain points, and only the top 10 get points). 'laps' refers to the number of labs completed. 'time' refers to the finishing time (in hour:minute:second:millisecond format), while 'milliseconds' only gives the finishing time in milliseconds. 'fastestLap' refers to the lap with the fastest time, and 'rank' refers to the fastest lap rank. 'fastestLapTime' refers to the time taken to complete the fastest lap, while 'fastestLapSpeed' refers to the speed (km/h) in the fastest lap. The 'statusId' is a foreign key link to the status table. 





To create the dataset, we merged four of our datasets: drivers, results, races, and circuits. This was to ensure that we would have the information on driver nationality, driver results for each race, the circuit where each race occurred, and the country where each circuit was. Before I started, I wanted to make a new variable, driver_country, in the driver dataset, which took the nationality variable and mapped it to its corresponding country. This was so the country where a driver is from could easily then be compared to the country where a race occurred. After I did this, I began the merging process. I first merged the drivers dataset to the results dataset on the 'driverId' variable, keeping 'nationality', and 'driver_country' from the driver dataset, and gaining 'raceId' and 'positionOrder' from the results dataset. I then merged the results dataset with the races dataset on the 'driverId' variable, gaining 'circuitId' from the races dataset. Finally, I merged the races dataset with the circuits dataset on the 'circuitId' variable, gaining 'country' from the races dataset. After that, I also looked at what which variable values were shared between the country variable (country where the circuit is) and the driver_country variable. I made a final_analysis_dataset based off this, so that we could look specifically at driver results for drivers that had at least once competed in their home country. 

In [21]:
# Cleaning our data

# View unique nationalities
unique_nationalities = drivers['nationality'].unique()
print(unique_nationalities)

# Create a mapping dictionary for nationalities to countries
nationality_to_country = {
    'British': 'United Kingdom',
    'German': 'Germany',
    'Spanish': 'Spain',
    'Finnish': 'Finland',
    'Japanese': 'Japan',
    'French': 'France',
    'Polish': 'Poland',
    'Brazilian': 'Brazil',
    'Italian': 'Italy',
    'Australian': 'Australia',
    'Austrian': 'Austria',
    'American': 'United States',
    'Dutch': 'Netherlands',
    'Colombian': 'Colombia',
    'Portuguese': 'Portugal',
    'Canadian': 'Canada',
    'Indian': 'India',
    'Hungarian': 'Hungary',
    'Irish': 'Ireland',
    'Danish': 'Denmark',
    'Argentine': 'Argentina',
    'Czech': 'Czech Republic',
    'Malaysian': 'Malaysia',
    'Swiss': 'Switzerland',
    'Belgian': 'Belgium',
    'Monegasque': 'Monaco',
    'Swedish': 'Sweden',
    'Venezuelan': 'Venezuela',
    'New Zealander': 'New Zealand',
    'Chilean': 'Chile',
    'Mexican': 'Mexico',
    'South African': 'South Africa',
    'Liechtensteiner': 'Liechtenstein',
    'Rhodesian': 'Zimbabwe',
    'American-Italian': 'United States', #not sure what to classify this as
    'Uruguayan': 'Uruguay',
    'Argentine-Italian': 'Italy', #not sure what to classify this as
    'Thai': 'Thailand',
    'East German': 'Germany',
    'Russian': 'Russia',
    'Indonesian': 'Indonesia',
    'Chinese': 'China'
}

# Map nationalities to countries
drivers['driver_country'] = drivers['nationality'].map(nationality_to_country)



#Merging our datasets

# Merge drivers with results on driverId, including nationality
merged_drivers_results = pd.merge(drivers[['driverId', 'nationality', 'driver_country']], results[['driverId', 'raceId', 'positionOrder']], on='driverId')

# Merge the resulting dataset with races on raceId
merged_results_races = pd.merge(merged_drivers_results, races[['raceId', 'circuitId']], on='raceId')

# Merge the resulting dataset with circuits on circuitId
final_dataset = pd.merge(merged_results_races, circuits[['circuitId', 'country']], on='circuitId')

# Display the resulting dataset
print(final_dataset[['driverId', 'raceId', 'positionOrder', 'circuitId', 'country', 'nationality']])


#Looking at shared values between the country and driver_country columns
shared_values = set(final_dataset['driver_country']).intersection(final_dataset['country'])
print(shared_values)

# Filter the final_dataset to include only rows where the country is in shared_values
final_analysis_dataset = final_dataset[final_dataset['country'].isin(shared_values)]

# Display the resulting dataset
print(final_analysis_dataset)


['British' 'German' 'Spanish' 'Finnish' 'Japanese' 'French' 'Polish'
 'Brazilian' 'Italian' 'Australian' 'Austrian' 'American' 'Dutch'
 'Colombian' 'Portuguese' 'Canadian' 'Indian' 'Hungarian' 'Irish' 'Danish'
 'Argentine' 'Czech' 'Malaysian' 'Swiss' 'Belgian' 'Monegasque' 'Swedish'
 'Venezuelan' 'New Zealander' 'Chilean' 'Mexican' 'South African'
 'Liechtensteiner' 'Rhodesian' 'American-Italian' 'Uruguayan'
 'Argentine-Italian' 'Thai' 'East German' 'Russian' 'Indonesian' 'Chinese']
       driverId  raceId  positionOrder  circuitId    country nationality
0             1      18              1          1  Australia     British
1             2      18              2          1  Australia      German
2             3      18              3          1  Australia      German
3             4      18              4          1  Australia     Spanish
4             5      18              5          1  Australia     Finnish
...         ...     ...            ...        ...        ...         ...
2